In [ ]:
!pip install koreanize-matplotlib #한글폰트 라이브러리 설치
!apt-get install -y fonts-nanum #나눔 고딕 폰트 설치
!pip install -q gdown

import gdown
import matplotlib.pyplot as plt
import koreanize_matplotlib  # 한글 자동 설정됨
import pandas as pd
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

# plt.rcParams['font.family'] = 'NanumGothic' #나눔 고딕 폰트 설정

In [ ]:
# CSV 파일 불러오기

transactions_file_id = '1rzQ8Bz6hGQ7IYZPWzEkaXIJXvA6pCFkq'
output_path = 'transactions_hm.csv'
gdown.download(f'https://drive.google.com/uc?id={transactions_file_id}', output_path, quiet=False)


articles = pd.read_csv('https://drive.google.com/uc?id=1OYMa1QBf6Lt2vVABrFJi8FO4UFhieVRv')
customers = pd.read_csv('https://drive.google.com/uc?id=1uyHiU-iKwJi8ht3TqBqmmFojvSc0CY_N')
transactions = pd.read_csv('transactions_hm.csv')

In [ ]:
# 각 테이블에 대한 열과 행 개수를 표현해주세요.
articles.shape # (105542, 25)
customers.shape # (1048575, 6)
transactions.shape # (1048575, 5)

In [ ]:
# 각 테이블에 대한 컬럼타입과 통계량을 보여주세요.
articles.info()
articles.describe()
customers.info()
customers.describe()
transactions.info()
transactions.describe()

In [ ]:
# NULL 값이 존재하는 경우, 대체 or 제거 모두 가능합니다.
customers.isnull().sum() # fashion_news_frequency (패션 뉴스 알람 주기) 컬럼 null 값 1개
articles.isnull().sum() # detail_desc (제품 상세 설명) 컬럼 null 값 416
transactions.isnull().sum() # null 값 0개

# NULL 값 제거/대체 필요없어 보임

In [ ]:
customers.isnull().sum()

# **가설 : 색상 전문 기업 Pantone의 2019년 year of color ‘Living Coral’ 색상별 매출에 영향을 미칠 것이다.**

In [ ]:
# 테이블을 결합하여, 데이터분석을 위한 하나의 데이터셋으로 만들어주세요.
# 모든 테이블을 결합하지 않아도 좋습니다.
# 테이블 결합 시 inner join 으로 진행해주세요.
df_total_table = pd.merge(transactions, customers, on='customer_id', how='inner')
df_total_table = pd.merge(df_total_table, articles, on='article_id', how='inner')
df_total_table.head()

In [ ]:
df_total_table["t_datetime"] = pd.to_datetime(df_total_table["t_dat"])
df_total_table["month"] = df_total_table["t_datetime"].dt.month

# 실제 금액처럼 보이게 price 정규화 해제 (예: 100,000 곱하기)
# df_total_table["price_real"] = df_total_table["price"] * 100000

# 월 별 판매량과 주문 수량 집계
df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 월 별 통계량
# df_total_table.groupby("month")["price"].agg(["mean", "median", "max", "min", "count"])

In [ ]:
import matplotlib.pyplot as plt

# 월별 집계
monthly_summary = df_total_table.groupby("month").agg(
    total_sales=("price", "sum"),
    order_count=("price", "count")
).sort_index()

# 통계값 계산
mean_sales = monthly_summary['total_sales'].mean()
median_sales = monthly_summary['total_sales'].median()
mean_orders = monthly_summary['order_count'].mean()
median_orders = monthly_summary['order_count'].median()

# 시각화 시작
fig, ax1 = plt.subplots(figsize=(12, 6))

# ① 실선: 판매 매출
ax1.plot(
    monthly_summary.index,
    monthly_summary['total_sales'],
    color='tab:blue',
    marker='o',
    linestyle='-',
    label='Total Sales'
)
ax1.axhline(mean_sales, color='tab:blue', linestyle=':', label='Sales Mean')

# ✅ 매출 데이터 라벨 (왼쪽으로 살짝 이동)
for x, y in zip(monthly_summary.index, monthly_summary['total_sales']):
    ax1.text(x - 0.15, y, f'{y:,.0f}', color='tab:blue', fontsize=9, ha='right', va='bottom')

ax1.set_xlabel('Month')
ax1.set_ylabel('Total Sales (₩)', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.set_xticks(monthly_summary.index)

# ② 실선: 주문량
ax2 = ax1.twinx()
ax2.plot(
    monthly_summary.index,
    monthly_summary['order_count'],
    color='tab:red',
    marker='s',
    linestyle='-',
    label='Order Count'
)
ax2.axhline(mean_orders, color='tab:red', linestyle=':', label='Order Mean')

# ✅ 주문 수 데이터 라벨 (오른쪽으로 살짝 이동)
for x, y in zip(monthly_summary.index, monthly_summary['order_count']):
    ax2.text(x + 0.15, y, f'{int(y)}', color='tab:red', fontsize=9, ha='left', va='bottom')

ax2.set_ylabel('Order Count', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# 제목
plt.title('월별 판매 매출 & 주문량 (평균선/중앙값 포함 + 좌우 분리 라벨)', fontsize=14)
fig.tight_layout()

# 범례 병합
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
fig.legend(lines1 + lines2, labels1 + labels2, loc='upper center', ncol=3)

plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 색상별 월별 판매량 집계
color_month_counts = df_total_table.groupby(['month', 'perceived_colour_master_name']).size().reset_index(name='count')

# 전체적으로 가장 많이 팔린 색상 Top 10
top_colors = color_month_counts.groupby('perceived_colour_master_name')['count'].sum().nlargest(10).index

# Top 10 색상만 필터링
filtered = color_month_counts[color_month_counts['perceived_colour_master_name'].isin(top_colors)]

# Pivot: 월을 index로, 색상별 컬럼으로
pivot_df = filtered.pivot(index='month', columns='perceived_colour_master_name', values='count').fillna(0).sort_index()


# 색상 이름 매핑 (matplotlib이 인식 가능한 이름 또는 HEX)
color_map = {
    'Red': 'red',
    'Blue': 'blue',
    'Green': 'green',
    'Black': 'black',
    'White': 'gray',
    'Yellow': 'yellow',
    'Pink': 'pink',
    'Orange': 'orange',
    'Khaki green': '#7e805d',
    'Beige': '#c8ad7f',
}

# NaN 처리
pivot_df = pivot_df.fillna(0)

plt.figure(figsize=(14, 8))

# 컬럼명 반복
for color in pivot_df.columns:
    line_color = color_map.get(color, 'gray')  # 매핑 없으면 회색
    try:
        plt.plot(pivot_df.index, pivot_df[color], marker='o', label=color, color=line_color)
    except Exception as e:
        print(f"Error plotting {color}: {e}")

plt.title('Top 10 색상별 월별 판매량 추이', fontsize=16)
plt.xlabel('월', fontsize=12)
plt.ylabel('판매 건수', fontsize=12)
plt.xticks(range(1, 13))
plt.legend(title='색상', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 계절 컬럼 생성
df_total_table['season'] = df_total_table['month'].map({
    12: '겨울', 1: '겨울', 2: '겨울',
    3: '봄', 4: '봄', 5: '봄',
    6: '여름', 7: '여름', 8: '여름',
    9: '가을', 10: '가을', 11: '가을'
})

# 색상별 계절별 판매량 집계
color_season_counts = df_total_table.groupby(['season', 'perceived_colour_master_name']).size().reset_index(name='count')

# Top 10 색상 기준 필터링
color_season_counts = color_season_counts[color_season_counts['perceived_colour_master_name'].isin(top_colors)]

# 피벗 테이블 생성
pivot_season = color_season_counts.pivot(index='season', columns='perceived_colour_master_name', values='count').fillna(0)

# 계절 순서 지정
season_order = ['봄', '여름', '가을', '겨울']
pivot_season = pivot_season.loc[season_order]


plt.figure(figsize=(12, 6))

for color in pivot_season.columns:
    line_color = color_map.get(color, 'gray')  # color_map에서 색상 가져오기, 없으면 회색
    plt.plot(pivot_season.index, pivot_season[color], marker='o', label=color, color=line_color)

plt.title('Top 10 색상별 계절별 판매량 추이', fontsize=16)
plt.xlabel('계절', fontsize=12)
plt.ylabel('판매 건수', fontsize=12)
plt.legend(title='색상', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 조합별 전체 구매 건수 집계
combo_count_total = df_total_table.groupby(
    ['sales_channel_id', 'product_type_name']
).size().reset_index(name='count')

# 채널 이름 매핑
channel_map = {1: '오프라인', 2: '온라인'}

# 최대 건수 구해서 x축 통일
max_count = combo_count_total['count'].max()

# Subplots 준비
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8), sharex=True)

# 그래프 그리기
for i, channel_id in enumerate([1, 2]):
    group = combo_count_total[combo_count_total['sales_channel_id'] == channel_id]
    top_combos = group.sort_values('count', ascending=False).head(10)

    sns.barplot(
        data=top_combos,
        y='product_type_name',
        x='count',
        palette='viridis',
        ax=axes[i]
    )

    axes[i].set_title(f"{channel_map[channel_id]} 채널", fontsize=14)
    axes[i].set_xlabel('Purchase Count')
    axes[i].set_ylabel('Product Type')
    axes[i].set_xlim(0, max_count * 1.1)  # 동일한 x축 범위

# 전체 타이틀과 레이아웃 조정
plt.suptitle('Top 10 Product Types by Purchase Count per Channel', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


# **가설 : 특정 기간(예: 월별)에 고객 활동량(거래 건수 또는 거래 금액)이 유의미한 차이를 보일 것이다.**

## **계절별-채널별 판매되는 상품 보기**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 계절, 채널 컬럼 생성
df_total_table['month'] = pd.to_datetime(df_total_table['t_dat']).dt.month
season_map = {12: '겨울', 1: '겨울', 2: '겨울',
              3: '봄', 4: '봄', 5: '봄',
              6: '여름', 7: '여름', 8: '여름',
              9: '가을', 10: '가을', 11: '가을'}
df_total_table['season'] = df_total_table['month'].map(season_map)

channel_map = {1: '오프라인', 2: '온라인'}
df_total_table['channel'] = df_total_table['sales_channel_id'].map(channel_map)

# 2. 계절 + 채널 + 상품유형별 판매량 집계
season_channel_product_counts = df_total_table.groupby(
    ['season', 'channel', 'product_type_name']
).size().reset_index(name='count')

# 3. 계절별, 채널별 Top 10 상품 뽑기
top10_season_channel_products = season_channel_product_counts.groupby(['season', 'channel']).apply(
    lambda x: x.nlargest(10, 'count')
).reset_index(drop=True)

# 4. 전체 기준으로 정렬 순서 지정
overall_order = top10_season_channel_products.groupby('product_type_name')['count'].sum().sort_values(ascending=False).index.tolist()

# 현재 데이터에 포함된 상품만 걸러서 순서 리스트 만들기
filtered_order = [ptype for ptype in overall_order if ptype in top10_season_channel_products['product_type_name'].unique()]

# 5. 그래프 그리기
g = sns.catplot(
    data=top10_season_channel_products,
    x='count',
    y='product_type_name',
    col='season',
    hue='channel',
    kind='bar',
    col_order=['봄', '여름', '가을', '겨울'],
    height=5,
    aspect=0.8,
    dodge=True,
    order=filtered_order
)

g.set_titles("{col_name} 판매량 Top 10")
g.set_axis_labels("판매 건수", "상품 유형")

# y축 글자 작게 (상품명이 길면 유용)
for ax in g.axes.flatten():
    ax.tick_params(axis='y', labelsize=8)

plt.tight_layout()
plt.show()


# **가설 : 연령대에 따라 총 매출 규모가 다를 것이다.**

In [ ]:
# 연령대 컬럼 생성
def get_age_group(age):
  if age < 20:
    return '10대 이하'
  elif age < 30:
    return '20대'
  elif age < 40:
    return '30대'
  elif age < 50:
    return '40대'
  elif age < 60:
    return '50대'
  else:
    return '60대 이상'

customers['age_group'] = customers['age'].apply(get_age_group)

In [ ]:
# 테이블을 결합하여, 데이터분석을 위한 하나의 데이터셋으로 만들어주세요.
# 모든 테이블을 결합하지 않아도 좋습니다.
# 테이블 결합 시 inner join 으로 진행해주세요.

# 거래 데이터에 고객 정보 병합
merged_df = pd.merge(transactions, customers, on='customer_id', how='inner')

In [ ]:
merged_df.head()

In [ ]:
# 연령대별 매출 집계
sales_by_age = merged_df.groupby('age_group')['price'].sum().reset_index()

sales_by_age

In [ ]:
plt.figure(figsize=(8,5))
bars = plt.bar(sales_by_age['age_group'], sales_by_age['price'])

# 막대 그래프 위에 매출 숫자 표시
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 50,  # 숫자가 막대 위로 조금 올라가게 조정
             f'{yval:,.0f}', ha='center', va='bottom')  # 숫자 형식을 쉼표로 추가

plt.title('연령대 별 매출 현황')
plt.xlabel('연령대')
plt.ylabel('총 매출')
plt.show()

In [ ]:
'''
분석 요약
1. 20대가 가장 큰 매출 비중을 차지하고 있다.
2. 그 다음은 30대, 50대, 40대가 많이 차지
3. 10대 이하는 매출 비중이 가장 낮다

'''

# **가설 : 연령대에 따라 구매 채널 선호도가 다르다**

In [ ]:
# 연령대 컬럼 생성
def get_age_group(age):
    if age < 20:
        return '10대 이하'
    elif age < 30:
        return '20대'
    elif age < 40:
        return '30대'
    elif age < 50:
        return '40대'
    elif age < 60:
        return '50대'
    else:
        return '60대 이상'


In [ ]:
customers['age_group'] = customers['age'].apply(get_age_group)

In [ ]:
customers.head()

In [ ]:
# 거래 데이터와 고객 데이터를 customer_id 기준으로 INNER JOIN
merged = pd.merge(transactions, customers, on='customer_id', how='inner')

In [ ]:
# 전체 고객 대상 온라인/오프라인 구매 비중
total_counts = merged['sales_channel_id'].value_counts().sort_index()
total_ratio = total_counts / total_counts.sum() * 100


print("전체 고객 대상 온라인/오프라인 구매 비중 (%)")
print(total_ratio)
print(total_counts)

# 온라인 비중이 더 높다

In [ ]:
# 파이 차트로 보기

ratio = total_ratio
labels = ['오프라인', '온라인']
explode = [0, 0.01]
wedgeprops={'width': 0.7, 'edgecolor': 'w', 'linewidth': 5}
plt.pie(ratio, labels=labels, autopct='%.1f%%', explode=explode, wedgeprops=wedgeprops)
plt.title('채널별 구매 비율')
plt.show()

In [ ]:
# 연령대별 온라인/오프라인 구매 비중
age_channel_counts = merged.groupby(['age_group', 'sales_channel_id']).size().unstack(fill_value=0)
age_channel_ratio = age_channel_counts.div(age_channel_counts.sum(axis=1), axis=0) * 100

print("\n연령대별 온라인/오프라인 구매 비중 (%)")
print(age_channel_ratio)

In [ ]:
# age_channel_ratio 재정리 (인덱스 초기화)
plot_df = age_channel_counts.reset_index()

# age_channel_counts 재정리 (인덱스 초기화)
plot_df = age_channel_counts.reset_index()

# 컬럼명 매핑 (1 → 오프라인, 2 → 온라인)
plot_df = plot_df.rename(columns={1: '오프라인', 2: '온라인'})

# Melt로 긴 형태로 변환 (seaborn barplot에 적합한 형태로 변환)
plot_df_melt = pd.melt(plot_df, id_vars='age_group', value_vars=['오프라인', '온라인'],
                       var_name='구매채널', value_name='매출')

# 시각화
plt.figure(figsize=(10,6))
ax = sns.barplot(x='age_group', y='매출', hue='구매채널', data=plot_df_melt)

# 막대 위에 매출 숫자 표시
for p in ax.patches:
    height = p.get_height()
    if height > 0:   # 막대의 높이 (매출 값)
        ax.text(p.get_x() + p.get_width() / 2, height + 1000,  # 숫자가 막대 위로 조금 올라가게 조정
            f'{height:,.0f}', ha='center', va='bottom')  # 매출 값 표시 (쉼표로 구분)

plt.title('연령대별 온라인/오프라인 매출 비교')
plt.ylabel('매출')
plt.xlabel('연령대')
plt.legend(title='구매채널')

plt.show()

In [ ]:
# 비중 재정리 (인덱스 초기화)
plot_df = age_channel_ratio.reset_index()

# 컬럼명 매핑 (1 → 오프라인, 2 → 온라인)
plot_df = plot_df.rename(columns={1: '오프라인', 2: '온라인'})

# Melt로 긴 형태로 변환 (seaborn barplot에 적합한 형태로 변환)
plot_df_melt = pd.melt(plot_df, id_vars='age_group', value_vars=['오프라인', '온라인'],
                       var_name='구매채널', value_name='비중')

# 시각화
plt.figure(figsize=(10,6))
ax = sns.barplot(x='age_group', y='비중', hue='구매채널', data=plot_df_melt)

# 막대 위에 비중 숫자 표시
for p in ax.patches:
    height = p.get_height()  # 막대의 높이 (비중 값)
    if height > 0:  # 0일 때는 표시하지 않음
       ax.text(p.get_x() + p.get_width() / 2, height + 1,  # 숫자가 막대 위로 조금 올라가게 조정
            f'{height:.1f}%', ha='center', va='bottom')  # 비중 값 표시 (소수점 1자리)

plt.title('연령대별 온라인/오프라인 구매 비율(%)')
plt.ylabel('비율 (%)')
plt.xlabel('연령대')
plt.legend(title='구매채널')
plt.ylim(0, 100)  # 비중을 퍼센트로 표시하기 위해 y축 범위 설정
plt.show()

# **가설 : 온라인/오프라인 각 채널에서 구매 집중되는 제품이 있을 것이다.**

In [ ]:
# 결측치 제거는 안하는 걸로

In [ ]:
# 거래 데이터에 상품 정보 병합
merged_df = pd.merge(transactions, articles, on = 'article_id', how = 'inner')

In [ ]:
# 채널별 카테고리 구매 비율 계산

# step 1: count 먼저
category_count = (
    merged_df.groupby(['sales_channel_id', 'product_type_name'])
    .size()
    .reset_index(name='count')
)

category_count

In [ ]:
# step 2: 비율(percentage) 계산
category_count['percentage'] = (
    category_count
    .groupby('sales_channel_id')['count']
    .transform(lambda x: 100 * x / x.sum())
)

category_count

In [ ]:
# 온라인 (sales_channel_id = 2) 상위 10개
top10_online = (
    category_count[category_count['sales_channel_id'] == 2]
    .sort_values(by='count', ascending=False)
    .head(10)
    .reset_index()
)
top10_online.index=top10_online.index+1
top10_online


In [ ]:
# 오프라인 (sales_channel_id = 1) 상위 10개
top10_offline = (
    category_count[category_count['sales_channel_id'] == 1]
    .sort_values(by='count', ascending=False)
    .head(10)
    .reset_index()
)
top10_offline.index=top10_offline.index+1

top10_offline

In [ ]:
# 공통 카테고리 기준으로 비교
top_categories = pd.concat([top10_online, top10_offline])
common_top = top_categories['product_type_name'].value_counts().index[:10]

online_data = top10_online[top10_online['product_type_name'].isin(common_top)][['product_type_name', 'count']]
offline_data = top10_offline[top10_offline['product_type_name'].isin(common_top)][['product_type_name', 'count']]

merged_counts = pd.merge(offline_data, online_data, on='product_type_name', how='outer', suffixes=('_offline', '_online')).fillna(0)
merged_counts = merged_counts.sort_values(by='product_type_name')

# 그래프
x = range(len(merged_counts))
plt.figure(figsize=(12, 6))
plt.bar([i - 0.2 for i in x], merged_counts['count_offline'], width=0.4, label='오프라인')
plt.bar([i + 0.2 for i in x], merged_counts['count_online'], width=0.4, label='온라인')
plt.xticks(x, merged_counts['product_type_name'], rotation=45, ha='right')
plt.ylabel('구매 건수')
plt.title('카테고리별 온라인 vs 오프라인 구매 건수 비교')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# 공통 카테고리
common_names = set(top10_online['product_type_name']) & set(top10_offline['product_type_name'])

# 색상 맵핑
all_names = pd.concat([top10_offline, top10_online])['product_type_name'].unique()
cmap = cm.get_cmap('tab20', len(all_names))

color_map = {}
used_index = 0
for name in all_names:
    if name in common_names:
        if name not in color_map:
            color_map[name] = cmap(used_index)
            used_index += 1
    else:
        color_map[name] = cmap(used_index)
        used_index += 1

# ✅ count 기준으로 정렬
top10_offline_sorted = top10_offline.sort_values(by='count', ascending=True)  # 오름차순으로 놓고 invert_yaxis()
top10_online_sorted = top10_online.sort_values(by='count', ascending=True)

# 그래프
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# 오프라인
axes[0].barh(top10_offline_sorted['product_type_name'], top10_offline_sorted['count'],
             color=[color_map[name] for name in top10_offline_sorted['product_type_name']])
axes[0].set_title('오프라인 Top 10')
axes[0].invert_yaxis()  # ✅ 가장 많은 게 위로 오도록 설정

# 온라인
axes[1].barh(top10_online_sorted['product_type_name'], top10_online_sorted['count'],
             color=[color_map[name] for name in top10_online_sorted['product_type_name']])
axes[1].set_title('온라인 Top 10')
axes[1].invert_yaxis()  # ✅ 역시 가장 많은 게 위

plt.suptitle('Top 10 카테고리 비교 (겹치는 항목은 같은 색)', fontsize=14)
plt.tight_layout()
plt.show()

# **가설 : 평일보다 주말에 매출이 높을 것이다**

## **요일별 매출 집계확인해보기**

In [ ]:
import pandas as pd

# 1. 데이터 불러오기
# 2. Null 값 제거
# 3. 날짜 컬럼을 datetime 형식으로 변환
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

# 4. 요일 정보 추출
# - weekday_num: 숫자 요일 (0=월, ..., 6=일)
# - weekday_name: 문자 요일 (Monday, ..., Sunday)
transactions['weekday_num'] = transactions['t_dat'].dt.dayofweek
transactions['weekday_name'] = transactions['t_dat'].dt.day_name()

# 5. 요일별 매출 집계 (총 매출 합계)
sales_by_weekday = (
    transactions.groupby('weekday_name')['price']
    .sum()
    .reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])  # 요일 순서대로 정렬
    .reset_index()
)

# 6. 결과 출력
print("📊 요일별 총 매출")
print(sales_by_weekday)

## **채널별-요일별 매출 확인해보기**

In [ ]:

# 날짜를 datetime으로 변환
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])

# 요일 컬럼 추가 (0=월 ~ 6=일 → 요일 이름으로 매핑)
weekday_names = ["월", "화", "수", "목", "금", "토", "일"]
transactions["weekday_name"] = transactions["t_dat"].dt.dayofweek.map(lambda x: weekday_names[x])

# 요일별 + 채널별 매출 집계
sales_by_weekday_named = (
    transactions
    .groupby(["weekday_name", "sales_channel_id"])["price"]
    .sum()
    .reset_index()
    .pivot(index="weekday_name", columns="sales_channel_id", values="price")
    .rename(columns={1: "오프라인", 2: "온라인"})
    .reindex(weekday_names)  # 요일 순서 유지
)

# 시각화
ax = sales_by_weekday_named.plot(kind="bar", figsize=(10, 8))
plt.title("요일별 채널별 매출 비교")
plt.xlabel("요일")
plt.ylabel("총 매출")
plt.xticks(rotation=0)
plt.legend(title="채널")
plt.tight_layout()

# 막대 위에 숫자 표시
for container in ax.containers:
    for bar in container:
        height = bar.get_height()
        if height > 0:  # 0 이상일 때만 표시
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height + (height * 0.01),  # 약간 위로
                f'{height:,.0f}',  # 천 단위 콤마, 정수로 표시
                ha='center', va='bottom', fontsize=9
            )

plt.show()

In [ ]:
# 요일별 + 채널별 구매 건수 집계
purchase_count_by_weekday = (
    transactions
    .groupby(["weekday_name", "sales_channel_id"])["article_id"]
    .count()
    .reset_index()
    .pivot(index="weekday_name", columns="sales_channel_id", values="article_id")
    .rename(columns={1: "오프라인", 2: "온라인"})
    .reindex(weekday_names)
)

# 시각화
ax = purchase_count_by_weekday.plot(kind="bar", figsize=(10, 8))
plt.title("요일별 채널별 구매 건수 비교")
plt.xlabel("요일")
plt.ylabel("구매 건 수")
plt.xticks(rotation=0)
plt.legend(title="채널")
plt.tight_layout()

# 막대 위에 숫자 표시
for container in ax.containers:
    for bar in container:
        height = bar.get_height()
        if height > 0:  # 0 이상만 표시
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height + (height * 0.01),  # 높이보다 약간 위
                f'{int(height):,}',        # 정수로 천 단위 콤마 표시
                ha='center', va='bottom', fontsize=9
            )

plt.show()